In [41]:
from polo2 import PoloDb
import numpy as np

In [3]:
polo_pub = "/Users/rca2t1/Dropbox/CODE/polo2-test/PUB"
db_file = f"{polo_pub}/aitf/aitf-mallet-trial1.db"
db = PoloDb(db_file)

In [90]:
word = db.get_table('word').set_index('word_id')
topicword = db.get_table('topicword').set_index(['word_id', 'topic_id'])
topic = db.get_table('topic').set_index('topic_id')

In [91]:
WTM = topicword.unstack(fill_value=0) + .001

In [92]:
PWT = WTM / WTM.sum()
PWT.columns = PWT.columns.droplevel(0)
PW = (WTM.T.sum() / WTM.T.sum().sum()).to_frame('p')

In [54]:
λ = .5

In [105]:
SIG = λ * np.log(PWT.T) + (1 - λ) * np.log(PWT.T/PW.p.T)

In [109]:
PHI = SIG.T.join(word.word_str).set_index('word_str')

In [115]:
PHI[].sort_values(ascending=False).head(10)

word_str
architecture    0.178104
units           0.087276
edge            0.066085
processor      -0.221514
analog         -0.282720
pim            -0.282720
isa            -0.282720
riscv          -0.282720
iot            -0.282720
constraints    -0.282720
Name: 3, dtype: float64